In [31]:
#Loading all the different files and merging them in one dataset 

import pandas as pd 
import os
import re 
import numpy as np

input_path = "./input_data/HDI/"
output_path = "./output_data/HDI"
os.makedirs(output_path, exist_ok=True)

dict_df = {}
for root,dirs,files in os.walk(input_path):
    for file in files:
        if file.endswith(".csv"):
            file_name = file[:-4]
            dict_df[file_name] = pd.read_csv(input_path + file)

In [32]:
#Check the years of the different files and delete those df with no years associated with their data 
df_with_no_year = {}
for key in list(dict_df.keys()):
    if('Unnamed: 3' in dict_df[key].columns):
        df_with_no_year[key] = dict_df[key]
        del dict_df[key]

In [33]:
#Joining all the dataframes in one dataframe with unified headers
all_data = pd.DataFrame({})
for k,v in dict_df.items():
    v['kpi_name'] = k
    v.drop(['Unnamed: 0', 'HDI Rank'], inplace=True , axis=1)
    v = v.melt(id_vars=['Country' , 'kpi_name'])
    v.columns = ['country_id' , 'kpi_name' , 'year' , 'value' ]
    all_data = all_data.append(v)

In [34]:
#Additional rows with null countries were found in thh data. From the source, we will remove those. 
#Checking for values with character and removing those characters 

all_data = all_data[all_data['country_id'].notnull()]
all_data['year'] = all_data['year'].astype(int)
all_data['value'] = all_data['value'].astype(str)
all_data["value"] = [''.join(re.findall("\d*\.?\d+", item)) for item in all_data["value"]]

In [35]:
all_data = pd.pivot_table(all_data , index=['kpi_name' , 'country_id'] , columns='year', aggfunc='first')
all_data = all_data.reset_index()
all_data.columns = all_data.columns.droplevel()
all_data.columns = ['kpi_name' , 'country'] + all_data.columns[2:].tolist()

In [36]:
#Saving Original file 
original_csv_name = output_path + "/HDI_complete.csv"
all_data.to_csv(original_csv_name , index=False)